In [21]:
import numpy as np
import scipy
from utils.SVD import svd_phaseI, svd_phaseIIA, svd, eigh_by_QR

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Problem 1: SVD by Two-Phase Approach

## Phase-I: Golub-Kahan Bidiagonalization

In [22]:
A = np.array([[1, 0, 1],
              [2, 5**.5, 0],
              [0, 0, 1],
              [0, 0, 1]])
B, Q, P = svd_phaseI(A)
print(B)


[[-2.23606798e+00  2.04939015e+00 -9.45175510e-17]
 [ 1.11022302e-16  8.39500986e-01 -5.07346289e-01]
 [ 0.00000000e+00 -2.67014343e-18 -1.19118383e+00]]


Todo: We may not need to form the reflection matrix H explicitly during Golub-Kahan bidiagonalization.

## Phase-II-A

In [23]:
T, Q = eigh_by_QR(B.T@B)
T, Q

(array([0.30237984, 1.73207775, 9.5466235 ]),
 array([[-0.68170526, -0.17908351,  0.70937087],
        [-0.69881931, -0.12770787, -0.70380557],
        [-0.21663221,  0.97551002,  0.03808789]]))

In [24]:
scipy.linalg.eigh(B.T@B)

(array([0.30237984, 1.73207775, 9.5466235 ]),
 array([[-0.68170526, -0.17908351,  0.70937087],
        [-0.69881931, -0.12770787, -0.70380557],
        [-0.21663221,  0.97551002,  0.03808789]]))

**Test SVD**

In [31]:
svd(A)

(array([[-0.61498971,  0.72937873,  0.21910421],
        [-0.36207877, -0.50746338,  0.77432904],
        [ 0.70049027,  0.37804764,  0.59260629]]),
 array([3.08976107, 1.31608425, 0.54989075]),
 array([[ 0.70937087, -0.17908351, -0.68170526],
        [ 0.69515536,  0.33750386,  0.63470479],
        [ 0.116413  , -0.92413216,  0.36390627]]))

In [32]:
scipy.linalg.svd(A, full_matrices=False)

(array([[-0.26444298,  0.60575682,  0.75041894],
        [-0.96312812, -0.2058522 , -0.17323137],
        [-0.03502958,  0.54345356, -0.45103343],
        [-0.03502958,  0.54345356, -0.45103343]]),
 array([3.09016724, 1.76483518, 0.5798476 ]),
 array([[-0.70892578, -0.69692666, -0.10824726],
        [ 0.10995498, -0.26081728,  0.95910596],
        [ 0.69665927, -0.66803262, -0.26153065]]))